# 🤖 Multi-Agent RAG System for Accounting Documents
This notebook implements a multi-agent Retrieval-Augmented Generation system tailored for accounting-related documents such as contracts, billing terms, and policies.

**Agents Involved:**
- Ingestion Agent – Loads and parses documents using SimpleDirectoryReader.
- Indexing Agent – Embeds documents and stores them in a Chroma vector database.
- Retrieval Agent – Handles user queries and fetches the most relevant document chunks.
- Summarization Agent – Formats and condenses answers using the selected LLM (e.g., GPT-3.5).
- Auditor Agent (Optional) – Provides transparency and traceability by explaining the source reasoning.

### 🔧 Stack
- `LlamaIndex` (document parsing, indexing)
- `OpenAI GPT-3.5` or `Mistral` for LLM
- `ChromaDB` for vector storage
- `LangChain` for agent orchestration
- `Gradio` UI for interaction


In [3]:
!pip uninstall -y llama-index

Found existing installation: llama-index 0.12.42
Uninstalling llama-index-0.12.42:
  Successfully uninstalled llama-index-0.12.42


In [1]:
# ✅ Install dependencies
# !pip install llama-index faiss-cpu openai langchain gradio chromadb tiktoken
!pip install llama-index-vector-stores-chroma

# Restart Kernel

In [2]:
# 📦 Imports
import os
import gradio as gr
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import openai

openai.api_key = os.getenv("OPENAI_API_KEY") or "your-api-key"

In [3]:
# 🧾 Agent 1: Ingest and parse documents
def ingest_docs():
    documents = SimpleDirectoryReader("./docs").load_data()
    return documents

In [6]:
# 🤖 Agent 2: Build Index (Chroma Store)
def build_index(documents):
    service_context = ServiceContext.from_defaults(
        llm=OpenAI(model="gpt-3.5-turbo"),
        embed_model=OpenAIEmbedding()
    )
    vector_store = ChromaVectorStore(persist_dir="./storage")
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    index = VectorStoreIndex.from_documents(
        documents,
        service_context=service_context,
        storage_context=storage_context
    )
    index.storage_context.persist()
    return index


In [7]:
# 🔍 Agent 3: Query the Index
def query_index(user_query):
    vector_store = ChromaVectorStore(persist_dir="./storage")
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.load_from_storage(storage_context)
    query_engine = index.as_query_engine()
    response = query_engine.query(user_query)
    return str(response)

In [9]:
# 🎛️ Gradio UI
gr.Interface(
    fn=query_index,
    inputs=gr.Textbox(label="Ask a question about accounting documents", lines=2),
    outputs=gr.Textbox(label="Answer", lines=10),
    title="📚 Multi-Agent RAG",
    description="Upload internal documents to ./docs and run the notebook to ingest."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8253374d5c85d90a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
